In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical

# --- Paths ---
test_paths = {
    "Coursera": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Coursera",
    "Programming": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Programming",
    "Youtube_Education": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Youtube",
    "Youtube_Entertainment": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Entertainment\Youtube",
    "Shopping": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Shopping"
}

# Updated model paths (excluding InceptionResNetV2)
model_paths = {
    "ResNet50": r"C:\Users\paray\OneDrive\Desktop\bits new research\final_global_resnet50_model.h5",
    "VGG19": r"C:\Users\paray\OneDrive\Desktop\bits new research\final_global_vgg19_model.h5",
    "VGG16": r"C:\Users\paray\OneDrive\Desktop\bits new research\final_trained_vgg16_model.h5",
    "Updated_ResNet50": r"C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_resnet50_model.h5"
}

# --- FL Parameters ---
num_clients = 5
epochs_per_client = 10
rounds_tfl = 5  # Traditional FL rounds
image_size = (224, 224)
num_classes = 5  # Number of categories

# Function to load test dataset
def load_test_data(test_paths, image_size=(224, 224)):
    images, labels = [], []
    label_map = {category: idx for idx, category in enumerate(test_paths.keys())}  # Assign labels
    
    for category, path in test_paths.items():
        for file in os.listdir(path):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(path, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label_map[category])
    
    return np.array(images), to_categorical(labels, num_classes)

# Load the test dataset
X_test, y_test = load_test_data(test_paths, image_size)

# --- Fixed Function for Computing FLOPs ---
def compute_flops(model):
    """
    Compute the number of Floating Point Operations (FLOPs) for a given model.
    """
    try:
        # Convert model to a concrete function
        concrete_func = tf.function(lambda inputs: model(inputs))
        concrete_func = concrete_func.get_concrete_function(
            tf.TensorSpec([1] + list(model.input_shape[1:]), model.input.dtype)
        )

        # Use TensorFlow Profiler
        run_meta = tf.compat.v1.RunMetadata()
        flops = tf.compat.v1.profiler.profile(
            graph=concrete_func.graph,
            run_meta=run_meta,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )

        return flops.total_float_ops if flops is not None else 0
    except Exception as e:
        print(f"Error computing FLOPs: {e}")
        return 0

# Function to calculate model size in MB
def model_size(model):
    param_size = sum([np.prod(var.shape) for var in model.trainable_weights]) * 4  # Convert to bytes
    return param_size / (1024 * 1024)  # Convert bytes to MB

# --- Compute Metrics for Each Model ---
results = {}

for model_name, model_path in model_paths.items():
    print(f"\nLoading model: {model_name} from {model_path}")
    model = load_model(model_path)  # Load pre-trained weights

    flops_per_epoch = compute_flops(model)
    model_size_mb = model_size(model)

    # Computational Cost
    osfl_total_flops = flops_per_epoch * epochs_per_client * num_clients
    tfl_total_flops = flops_per_epoch * epochs_per_client * num_clients * rounds_tfl

    # Communication Overhead
    osfl_comm_overhead = model_size_mb * num_clients  # Model sent once per client
    tfl_comm_overhead = model_size_mb * num_clients * rounds_tfl  # Model sent every round

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

    results[model_name] = {
        "Model Size (MB)": model_size_mb,
        "FLOPs per Epoch": flops_per_epoch,
        "OSFL Total FLOPs": osfl_total_flops,
        "TFL Total FLOPs (5 rounds)": tfl_total_flops,
        "OSFL Communication Overhead (MB)": osfl_comm_overhead,
        "TFL Communication Overhead (MB)": tfl_comm_overhead,
        "Test Accuracy": test_accuracy,
    }

# --- Print Final Results ---
print("\n--- Computational Cost, Communication Overhead & Accuracy for All Models ---")
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"  Model Size: {metrics['Model Size (MB)']:.2f} MB")
    print(f"  FLOPs per Epoch: {metrics['FLOPs per Epoch']:,}")
    print(f"  One-Shot FL - Total FLOPs: {metrics['OSFL Total FLOPs']:,}")
    print(f"  Traditional FL (5 Rounds) - Total FLOPs: {metrics['TFL Total FLOPs (5 rounds)']:,}")
    print(f"  One-Shot FL Communication Overhead: {metrics['OSFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Traditional FL Communication Overhead: {metrics['TFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Test Accuracy: {metrics['Test Accuracy']:.2%}")



Loading model: ResNet50 from C:\Users\paray\OneDrive\Desktop\bits new research\final_global_resnet50_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.7697 - loss: 0.6666

Loading model: VGG19 from C:\Users\paray\OneDrive\Desktop\bits new research\final_global_vgg19_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 5s/step - accuracy: 0.8834 - loss: 0.3504

Loading model: VGG16 from C:\Users\paray\OneDrive\Desktop\bits new research\final_trained_vgg16_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.8236 - loss: 0.4707

Loading model: Updated_ResNet50 from C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_resnet50_model.h5


4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.7697 - loss: 0.6666

--- Computational Cost, Communication Overhead & Accuracy for All Models ---

Model: ResNet50
  Model Size: 0.04 MB
  FLOPs per Epoch: 7,751,217,822
  One-Shot FL - Total FLOPs: 387,560,891,100
  Traditional FL (5 Rounds) - Total FLOPs: 1,937,804,455,500
  One-Shot FL Communication Overhead: 0.20 MB
  Traditional FL Communication Overhead: 0.98 MB
  Test Accuracy: 73.68%

Model: VGG19
  Model Size: 0.01 MB
  FLOPs per Epoch: 39,037,861,406
  One-Shot FL - Total FLOPs: 1,951,893,070,300
  Traditional FL (5 Rounds) - Total FLOPs: 9,759,465,351,500
  One-Shot FL Communication Overhead: 0.05 MB
  Traditional FL Communication Overhead: 0.24 MB
  Test Accuracy: 85.96%

Model: VGG16
  Model Size: 0.01 MB
  FLOPs per Epoch: 30,712,960,542
  One-Shot FL - Total FLOPs: 1,535,648,027,100
  Traditional FL (5 Rounds) - Total FLOPs: 7,678,240,135,500
  One-Shot FL Communication Overhead: 0.05 MB
  Traditional FL Communication Ove

In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Layer

# --- Custom Layer Definition ---
class CustomScaleLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomScaleLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return inputs * 0.1  # Example scaling operation

    def get_config(self):
        config = super().get_config()
        return config


# --- Paths ---
test_paths = {
    "Coursera": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Coursera",
    "Programming": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Programming",
    "Youtube_Education": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Youtube",
    "Youtube_Entertainment": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Entertainment\Youtube",
    "Shopping": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Shopping"
}

# --- Model Paths ---
model_paths = {
    "Updated_InceptionResNetV2": r"C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_inceptionresnetv2_model.h5"
}

# --- FL Parameters ---
num_clients = 5
epochs_per_client = {
    "Updated_InceptionResNetV2": 10,
}
rounds_tfl = 5  # Traditional FL rounds
image_size = (299, 299)  # Resize to 299x299 for InceptionResNetV2
num_classes = 5  # Number of categories


# Function to load test dataset
def load_test_data(test_paths, image_size=(299, 299)):
    images, labels = [], []
    label_map = {category: idx for idx, category in enumerate(test_paths.keys())}  # Assign labels
    
    for category, path in test_paths.items():
        for file in os.listdir(path):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(path, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label_map[category])
    
    return np.array(images), to_categorical(labels, num_classes)


# Load the test dataset
X_test, y_test = load_test_data(test_paths, image_size)


# --- Function to Compute FLOPs ---
def compute_flops(model):
    """
    Compute the number of Floating Point Operations (FLOPs) for a given model.
    """
    try:
        # Convert model to a concrete function
        concrete_func = tf.function(lambda inputs: model(inputs))
        concrete_func = concrete_func.get_concrete_function(
            tf.TensorSpec([1] + list(model.input_shape[1:]), model.input.dtype)
        )

        # Use TensorFlow Profiler
        run_meta = tf.compat.v1.RunMetadata()
        flops = tf.compat.v1.profiler.profile(
            graph=concrete_func.graph,
            run_meta=run_meta,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )

        return flops.total_float_ops if flops is not None else 0
    except Exception as e:
        print(f"Error computing FLOPs: {e}")
        return 0


# Function to calculate model size in MB
def model_size(model):
    param_size = sum([np.prod(var.shape) for var in model.trainable_weights]) * 4  # Convert to bytes
    return param_size / (1024 * 1024)  # Convert bytes to MB


# --- Compute Metrics for InceptionResNetV2 ---
results = {}

# Register the custom layer
custom_objects = {"CustomScaleLayer": CustomScaleLayer}

model_name = "Updated_InceptionResNetV2"
model_path = model_paths[model_name]

print(f"\nLoading model: {model_name} from {model_path}")
try:
    model = load_model(model_path, custom_objects=custom_objects)  # Load pre-trained weights
    print(f"Successfully loaded {model_name}")
except Exception as e:
    print(f"Error loading {model_name}: {e}")
    exit(1)  # Exit if loading fails

# Compile the model (needed before evaluation)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Compute metrics
flops_per_epoch = compute_flops(model)
model_size_mb = model_size(model)
epochs = epochs_per_client[model_name]

# Computational Cost
osfl_total_flops = flops_per_epoch * epochs * num_clients
tfl_total_flops = flops_per_epoch * epochs * num_clients * rounds_tfl

# Communication Overhead
osfl_comm_overhead = model_size_mb * num_clients  # Model sent once per client
tfl_comm_overhead = model_size_mb * num_clients * rounds_tfl  # Model sent every round

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

results[model_name] = {
    "Model Size (MB)": model_size_mb,
    "FLOPs per Epoch": flops_per_epoch,
    "OSFL Total FLOPs": osfl_total_flops,
    "TFL Total FLOPs (5 rounds)": tfl_total_flops,
    "OSFL Communication Overhead (MB)": osfl_comm_overhead,
    "TFL Communication Overhead (MB)": tfl_comm_overhead,
    "Test Accuracy": test_accuracy,
}

# --- Print Final Results ---
print("\n--- Computational Cost, Communication Overhead & Accuracy for InceptionResNetV2 ---")
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"  Model Size: {metrics['Model Size (MB)']:.2f} MB")
    print(f"  FLOPs per Epoch: {metrics['FLOPs per Epoch']:,}")
    print(f"  One-Shot FL - Total FLOPs: {metrics['OSFL Total FLOPs']:,}")
    print(f"  Traditional FL (5 Rounds) - Total FLOPs: {metrics['TFL Total FLOPs (5 rounds)']:,}")
    print(f"  One-Shot FL Communication Overhead: {metrics['OSFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Traditional FL Communication Overhead: {metrics['TFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Test Accuracy: {metrics['Test Accuracy']:.2%}")



Loading model: Updated_InceptionResNetV2 from C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_inceptionresnetv2_model.h5
Error loading Updated_InceptionResNetV2: Error when deserializing class 'CustomScaleLayer' using config={'name': 'custom_scale_layer_120', 'trainable': False, 'dtype': 'float32', 'scale': 0.17}.

Exception encountered: Unrecognized keyword arguments passed to CustomScaleLayer: {'scale': 0.17}
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8508 - loss: 0.5633

--- Computational Cost, Communication Overhead & Accuracy for InceptionResNetV2 ---

Model: Updated_InceptionResNetV2
  Model Size: 0.04 MB
  FLOPs per Epoch: 11,464,908,190
  One-Shot FL - Total FLOPs: 573,245,409,500
  Traditional FL (5 Rounds) - Total FLOPs: 2,866,227,047,500
  One-Shot FL Communication Overhead: 0.20 MB
  Traditional FL Communication Overhead: 0.98 MB
  Test Accuracy: 85.09%


244 X 244

In [20]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Layer

# --- Custom Layer Definition ---
class CustomScaleLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomScaleLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return inputs * 0.1  # Example scaling operation

    def get_config(self):
        config = super().get_config()
        return config


# --- Paths ---
test_paths = {
    "Coursera": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Coursera",
    "Programming": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Programming",
    "Youtube_Education": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Youtube",
    "Youtube_Entertainment": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Entertainment\Youtube",
    "Shopping": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Shopping"
}

# --- Model Paths ---
model_paths = {
    
    "Updated_MobileNet": r"C:\Users\paray\OneDrive\Desktop\bits new research\updated_mobilenet_model.h5"
}

# --- FL Parameters ---
num_clients = 5
epochs_per_client = {
    "Updated_InceptionV3": 15,
    "Updated_MobileNet": 18
}
rounds_tfl = 5  # Traditional FL rounds
image_size = (224, 224)  # Resize to 224x224 for InceptionV3 and MobileNet
num_classes = 5  # Number of categories


# Function to load test dataset
def load_test_data(test_paths, image_size=(224, 224)):
    images, labels = [], []
    label_map = {category: idx for idx, category in enumerate(test_paths.keys())}  # Assign labels
    
    for category, path in test_paths.items():
        for file in os.listdir(path):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(path, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label_map[category])
    
    return np.array(images), to_categorical(labels, num_classes)


# Load the test dataset
X_test, y_test = load_test_data(test_paths, image_size)


# --- Function to Compute FLOPs ---
def compute_flops(model):
    """
    Compute the number of Floating Point Operations (FLOPs) for a given model.
    """
    try:
        # Convert model to a concrete function
        concrete_func = tf.function(lambda inputs: model(inputs))
        concrete_func = concrete_func.get_concrete_function(
            tf.TensorSpec([1] + list(model.input_shape[1:]), model.input.dtype)
        )

        # Use TensorFlow Profiler
        run_meta = tf.compat.v1.RunMetadata()
        flops = tf.compat.v1.profiler.profile(
            graph=concrete_func.graph,
            run_meta=run_meta,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )

        return flops.total_float_ops if flops is not None else 0
    except Exception as e:
        print(f"Error computing FLOPs: {e}")
        return 0


# Function to calculate model size in MB
def model_size(model):
    param_size = sum([np.prod(var.shape) for var in model.trainable_weights]) * 4  # Convert to bytes
    return param_size / (1024 * 1024)  # Convert bytes to MB


# --- Compute Metrics for InceptionV3 and MobileNet ---
results = {}

# Register the custom layer
custom_objects = {"CustomScaleLayer": CustomScaleLayer}

for model_name, model_path in model_paths.items():
    print(f"\nLoading model: {model_name} from {model_path}")
    try:
        model = load_model(model_path, custom_objects=custom_objects)  # Load pre-trained weights
        print(f"Successfully loaded {model_name}")
    except Exception as e:
        print(f"Error loading {model_name}: {e}")
        continue  # Skip this model if loading fails

    # Compile the model (needed before evaluation)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Compute metrics
    flops_per_epoch = compute_flops(model)
    model_size_mb = model_size(model)
    epochs = epochs_per_client[model_name]

    # Computational Cost
    osfl_total_flops = flops_per_epoch * epochs * num_clients
    tfl_total_flops = flops_per_epoch * epochs * num_clients * rounds_tfl

    # Communication Overhead
    osfl_comm_overhead = model_size_mb * num_clients  # Model sent once per client
    tfl_comm_overhead = model_size_mb * num_clients * rounds_tfl  # Model sent every round

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

    results[model_name] = {
        "Model Size (MB)": model_size_mb,
        "FLOPs per Epoch": flops_per_epoch,
        "OSFL Total FLOPs": osfl_total_flops,
        "TFL Total FLOPs (5 rounds)": tfl_total_flops,
        "OSFL Communication Overhead (MB)": osfl_comm_overhead,
        "TFL Communication Overhead (MB)": tfl_comm_overhead,
        "Test Accuracy": test_accuracy,
    }

# --- Print Final Results ---
print("\n--- Computational Cost, Communication Overhead & Accuracy for InceptionV3 and MobileNet ---")
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"  Model Size: {metrics['Model Size (MB)']:.2f} MB")
    print(f"  FLOPs per Epoch: {metrics['FLOPs per Epoch']:,}")
    print(f"  One-Shot FL - Total FLOPs: {metrics['OSFL Total FLOPs']:,}")
    print(f"  Traditional FL (5 Rounds) - Total FLOPs: {metrics['TFL Total FLOPs (5 rounds)']:,}")
    print(f"  One-Shot FL Communication Overhead: {metrics['OSFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Traditional FL Communication Overhead: {metrics['TFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Test Accuracy: {metrics['Test Accuracy']:.2%}")



Loading model: Updated_MobileNet from C:\Users\paray\OneDrive\Desktop\bits new research\updated_mobilenet_model.h5


Successfully loaded Updated_MobileNet
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 406ms/step - accuracy: 0.9044 - loss: 0.4044

--- Computational Cost, Communication Overhead & Accuracy for InceptionV3 and MobileNet ---

Model: Updated_MobileNet
  Model Size: 0.02 MB
  FLOPs per Epoch: 1,145,644,190
  One-Shot FL - Total FLOPs: 103,107,977,100
  Traditional FL (5 Rounds) - Total FLOPs: 515,539,885,500
  One-Shot FL Communication Overhead: 0.10 MB
  Traditional FL Communication Overhead: 0.49 MB
  Test Accuracy: 88.60%


In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Layer

# --- Custom Layer Definition ---
class CustomScaleLayer(Layer):
    def __init__(self, **kwargs):
        super(CustomScaleLayer, self).__init__(**kwargs)

    def call(self, inputs):
        return inputs * 0.1  # Example scaling operation

    def get_config(self):
        config = super().get_config()
        return config


# --- Paths ---
test_paths = {
    "Coursera": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Coursera",
    "Programming": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Programming",
    "Youtube_Education": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Education\Youtube",
    "Youtube_Entertainment": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Entertainment\Youtube",
    "Shopping": r"C:\Users\paray\OneDrive\Desktop\bits new research\test\Shopping"
}

# --- Model Paths ---
model_paths = {
    "Updated_InceptionV3": r"C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_inceptionv3_model.h5"
}

# --- FL Parameters ---
num_clients = 5
epochs_per_client = {
    "Updated_InceptionV3": 15
}
rounds_tfl = 5  # Traditional FL rounds
image_size = (299, 299)  # Resize to 299x299 for InceptionV3
num_classes = 5  # Number of categories


# Function to load test dataset
def load_test_data(test_paths, image_size=(299, 299)):
    images, labels = [], []
    label_map = {category: idx for idx, category in enumerate(test_paths.keys())}  # Assign labels
    
    for category, path in test_paths.items():
        for file in os.listdir(path):
            if file.endswith(".jpg") or file.endswith(".png"):
                img_path = os.path.join(path, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img) / 255.0  # Normalize
                images.append(img_array)
                labels.append(label_map[category])
    
    return np.array(images), to_categorical(labels, num_classes)


# Load the test dataset
X_test, y_test = load_test_data(test_paths, image_size)


# --- Function to Compute FLOPs ---
def compute_flops(model):
    """
    Compute the number of Floating Point Operations (FLOPs) for a given model.
    """
    try:
        # Convert model to a concrete function
        concrete_func = tf.function(lambda inputs: model(inputs))
        concrete_func = concrete_func.get_concrete_function(
            tf.TensorSpec([1] + list(model.input_shape[1:]), model.input.dtype)
        )

        # Use TensorFlow Profiler
        run_meta = tf.compat.v1.RunMetadata()
        flops = tf.compat.v1.profiler.profile(
            graph=concrete_func.graph,
            run_meta=run_meta,
            options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        )

        return flops.total_float_ops if flops is not None else 0
    except Exception as e:
        print(f"Error computing FLOPs: {e}")
        return 0


# Function to calculate model size in MB
def model_size(model):
    param_size = sum([np.prod(var.shape) for var in model.trainable_weights]) * 4  # Convert to bytes
    return param_size / (1024 * 1024)  # Convert bytes to MB


# --- Compute Metrics for InceptionV3 ---
results = {}

# Register the custom layer
custom_objects = {"CustomScaleLayer": CustomScaleLayer}

for model_name, model_path in model_paths.items():
    print(f"\nLoading model: {model_name} from {model_path}")
    try:
        model = load_model(model_path, custom_objects=custom_objects)  # Load pre-trained weights
        print(f"Successfully loaded {model_name}")
    except Exception as e:
        print(f"Error loading {model_name}: {e}")
        continue  # Skip this model if loading fails

    # Compile the model (needed before evaluation)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Compute metrics
    flops_per_epoch = compute_flops(model)
    model_size_mb = model_size(model)
    epochs = epochs_per_client[model_name]

    # Computational Cost
    osfl_total_flops = flops_per_epoch * epochs * num_clients
    tfl_total_flops = flops_per_epoch * epochs * num_clients * rounds_tfl

    # Communication Overhead
    osfl_comm_overhead = model_size_mb * num_clients  # Model sent once per client
    tfl_comm_overhead = model_size_mb * num_clients * rounds_tfl  # Model sent every round

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)

    results[model_name] = {
        "Model Size (MB)": model_size_mb,
        "FLOPs per Epoch": flops_per_epoch,
        "OSFL Total FLOPs": osfl_total_flops,
        "TFL Total FLOPs (5 rounds)": tfl_total_flops,
        "OSFL Communication Overhead (MB)": osfl_comm_overhead,
        "TFL Communication Overhead (MB)": tfl_comm_overhead,
        "Test Accuracy": test_accuracy,
    }

# --- Print Final Results ---
print("\n--- Computational Cost, Communication Overhead & Accuracy for InceptionV3 ---")
for model_name, metrics in results.items():
    print(f"\nModel: {model_name}")
    print(f"  Model Size: {metrics['Model Size (MB)']:.2f} MB")
    print(f"  FLOPs per Epoch: {metrics['FLOPs per Epoch']:,}")
    print(f"  One-Shot FL - Total FLOPs: {metrics['OSFL Total FLOPs']:,}")
    print(f"  Traditional FL (5 Rounds) - Total FLOPs: {metrics['TFL Total FLOPs (5 rounds)']:,}")
    print(f"  One-Shot FL Communication Overhead: {metrics['OSFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Traditional FL Communication Overhead: {metrics['TFL Communication Overhead (MB)']:.2f} MB")
    print(f"  Test Accuracy: {metrics['Test Accuracy']:.2%}")



Loading model: Updated_InceptionV3 from C:\Users\paray\OneDrive\Desktop\bits new research\updated_global_inceptionv3_model.h5


Successfully loaded Updated_InceptionV3
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8508 - loss: 0.5633

--- Computational Cost, Communication Overhead & Accuracy for InceptionV3 ---

Model: Updated_InceptionV3
  Model Size: 0.04 MB
  FLOPs per Epoch: 11,464,908,190
  One-Shot FL - Total FLOPs: 859,868,114,250
  Traditional FL (5 Rounds) - Total FLOPs: 4,299,340,571,250
  One-Shot FL Communication Overhead: 0.20 MB
  Traditional FL Communication Overhead: 0.98 MB
  Test Accuracy: 85.09%
